In [4]:
using DelimitedFiles: readdlm
using Plots
using PyPlot
using Images
using ImageView: imshow
using ImageMagick
using PyCall

In [2]:
@doc Images.load

  * `load(filename)` loads the contents of a formatted file, trying to infer

the format from `filename` and/or magic bytes in the file.

  * `load(strm)` loads from an `IOStream` or similar object. In this case,

there is no filename extension, so we rely on the magic bytes for format identification.

  * `load(File(format"PNG", filename))` specifies the format directly, and bypasses inference.
  * `load(Stream(format"PNG", io))` specifies the format directly, and bypasses inference.
  * `load(f; options...)` passes keyword arguments on to the loader.


In [6]:
@doc Images.imresize

```
imresize(img, sz) -> imgr
imresize(img, inds) -> imgr
imresize(img; ratio) -> imgr
```

Change `img` to be of size `sz` (or to have indices `inds`). If `ratio` is used, then `sz = ceil(Int, size(img).*ratio)`. This interpolates the values at sub-pixel locations. If you are shrinking the image, you risk aliasing unless you low-pass filter `img` first.

# Examples

```julia
julia> img = testimage("lena_gray_256") # 256*256
julia> imresize(img, 128, 128) # 128*128
julia> imresize(img, 1:128, 1:128) # 128*128
julia> imresize(img, (128, 128)) # 128*128
julia> imresize(img, (1:128, 1:128)) # 128*128
julia> imresize(img, (1:128, )) # 128*256
julia> imresize(img, 128) # 128*256
julia> imresize(img, ratio = 0.5) # 
julia> imresize(img, ratio = (2, 1)) # 256*128

σ = map((o,n)->0.75*o/n, size(img), sz)
kern = KernelFactors.gaussian(σ)   # from ImageFiltering
imgr = imresize(imfilter(img, kern, NA()), sz)
```

See also [`restrict`](@ref).


In [14]:
@doc ImageView.imshow

```
imshow()
```

Choose an image to display via a file dialog.

```
imshow(img; axes=(1,2), name="ImageView") -> guidict
imshow(img, clim; axes=(1,2), name="ImageView") -> guidict
imshow(img, clim, zoomregion, slicedata, annotations; axes=(1,2), name="ImageView") -> guidict
```

Display the image `img` in a new window titled with `name`, returning a dictionary `guidict` containing any Reactive signals or GtkReactive widgets. If the image is 3 or 4 dimensional, GUI controls will be added for slicing along "extra" axes. By default the two-dimensional slice containing axes 1 and 2 are shown, but that can be changed by passing a different setting for `axes`.

If the image is grayscale, by default contrast is set by a `scaleminmax` object whose end-points can be modified by right-clicking on the image. If `clim == nothing`, the image's own native contrast is used (`clamp01nan`).  You may also pass a custom contrast function.

Finally, you may specify [`GtkReactive.ZoomRegion`](@ref) and [`SliceData`](@ref) signals. See also [`roi`](@ref), as well as any `annotations` that you wish to apply.

```
imshow(canvas, imgsig::Signal) -> guidict
imshow(canvas, imgsig::Signal, zr::Signal{ZoomRegion}) -> guidict
imshow(frame::Frame, canvas, imgsig::Signal, zr::Signal{ZoomRegion}) -> guidict
```

Display `imgsig` (a `Signal` of an image) in `canvas`, setting up panning and zooming. Optionally include a `frame` for preserving aspect ratio. `imgsig` must be two-dimensional (but can be a Signal-view of a higher-dimensional object).

# Example

```julia
using ImageView, TestImages, Gtk
mri = testimage("mri");
# Create a canvas `c`. There are other approaches, like stealing one from a previous call
# to `imshow`, or using GtkReactive directly.
guidict = imshow_gui((300, 300))
c = guidict["canvas"];
# To see anything you have to call `showall` on the window (once)
Gtk.showall(guidict["window"])
# Create the image Signal
imgsig = Signal(mri[:,:,1]);
# Show it
imshow(c, imgsig)
# Now anytime you want to update, just push! a new image
push!(imgsig, mri[:,:,8])
```


In [20]:
img = load("IMG_5529.jpg")
@show size(img)

size(img) = (3264, 2448)


(3264, 2448)

In [22]:
img_resized = Images.imresize(img, ratio = 0.5)
@show size(img_resized)

size(img_resized) = (1632, 1224)


(1632, 1224)